In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import lofargeo

In [4]:
import glob

In [5]:
data = pd.DataFrame();
for stationfile in glob.glob('/Users/dijkema/opt/lofar/src/MAC/Deployment/data/Coordinates/ETRF_FILES/*/*.csv'):
    stationframe = pd.read_csv(stationfile)
    stationframe["STATIONNAME"] = stationfile.split("/")[-2]
    data = pd.concat([data, stationframe], ignore_index=True)

In [6]:
data["NAME"].str[0];

In [7]:
def gettype(nameincsv):
    if not nameincsv.strip()[1].isnumeric():
        return nameincsv.strip()
    else:
        return nameincsv.strip()[0]+"BA"

In [8]:
def getnum(nameincsv):
    if not nameincsv.strip()[1].isnumeric():
        return -1
    else:
        return int(nameincsv.strip()[1:])

In [9]:
data["TYPE"] = data.apply(lambda row: gettype(row["NAME"]), axis=1)

In [10]:
data["NUMBER"] = data.apply(lambda row: getnum(row["NAME"]), axis=1)

In [11]:
for coordpart in ['lon_rad', 'lat_rad', 'height_m']:
    data["ETRS_"+coordpart.upper().split("_")[0]] = \
          data.apply(lambda row: 
                       np.rad2deg(
                         lofargeo.geographic_from_xyz(
                           (row['ETRS-X'],row['ETRS-Y'],row['ETRS-Z'])
                         )[coordpart]
                       ), axis=1)

In [12]:
def getcoord(stationname, num):
    """
    Get the lat-lon coordinates of an HBA-antenna
    """
    queryres = data[(data["STATIONNAME"]==stationname) & (data["NUMBER"]==num) & (data["TYPE"]=="HBA")]
    if len(queryres) != 1:
        raise Exception("Problem finding antenna "+str(num)+" for station"+stationname+": found "+len(queryres))
    return np.array([queryres["ETRS_LON"].iloc[0], queryres["ETRS_LAT"].iloc[0]])

Get the x and y direction in a station

In [13]:
def numbelowfirst(stationtype):
    """ Get the antenna number for when the number in a line is below zero.
    Stationtype must be CS, RS or other"""
    if stationtype=='CSHBA0':
        return 3
    if stationtype=='CSHBA1':
        return 27
    if stationtype=='RS':
        return 4
    else:
        return 6

In [14]:
dirxs = {}
dirys = {}
for stationname in pd.unique(data["STATIONNAME"]):
    subtypes = [""]
    if stationname[0:2] == "CS":
        subtypes = ["HBA0", "HBA1"]
    for subtype in subtypes:
        firstantenna = 0
        if subtype == "HBA1":
            firstantenna = 24;
        ant0coord = getcoord(stationname, firstantenna)
        dirxs[stationname+subtype] = getcoord(stationname, firstantenna+1) - ant0coord
        highy = numbelowfirst(stationname[0:2]+subtype)
        dirys[stationname+subtype] = getcoord(stationname, highy) - ant0coord

In [15]:
data["ETRS_WKT"] = ""
for index, rec in data.iterrows():
    if rec['TYPE']!='HBA':
        continue
        
    xy=np.array([rec['ETRS_LON'],rec['ETRS_LAT']])
    
    # Find the offset vector between tiles
    subtype = ""
    if rec['STATIONNAME'][0:2] == "CS":
        if rec['NUMBER'] >= 24:
            subtype = "HBA1"
        else:
            subtype = "HBA0"
            
    dirx = dirxs[rec['STATIONNAME'] + subtype]
    diry = dirys[rec['STATIONNAME'] + subtype]

    ul = xy - 0.5*dirx + 0.5*diry
    ur = xy + 0.5*dirx + 0.5*diry 
    lr = xy + 0.5*dirx - 0.5*diry 
    ll = xy - 0.5*dirx - 0.5*diry 

    wkt=('Polygon (('+str(ul[0])+' '+str(ul[1])+', '+ 
                      str(ur[0])+' '+str(ur[1])+', '+ \
                      str(lr[0])+' '+str(lr[1])+', '+ \
                      str(ll[0])+' '+str(ll[1])+', '+ \
                      str(ul[0])+' '+str(ul[1])+'))')
    
    data.set_value(index, "ETRS_WKT", wkt)

In [16]:
a=data[data["ETRS_WKT"]!=""]["ETRS_WKT"].iloc[0]

In [17]:
a[10:].split(', ')

['6.86861583228 52.9117170256',
 '6.86868577076 52.9116982185',
 '6.86871689402 52.9117404904',
 '6.86864695554 52.9117592976',
 '6.86861583228 52.9117170256))']

In [20]:
data.to_csv("allantennas.csv")

In [22]:
data.iloc[0]

NAME                    L0
ETRS-X         3.82692e+06
ETRS-Y              460915
ETRS-Z         5.06464e+06
STATION-P                0
STATION-Q                0
STATION-R                0
RCU-X                    0
RCU-Y                    1
STATIONNAME          CS001
TYPE                   LBA
NUMBER                   0
ETRS_LON           6.86763
ETRS_LAT           52.9114
ETRS_HEIGHT        2874.11
ETRS_WKT                  
Name: 0, dtype: object